In [ ]:
%matplotlib inline
from os import listdir
from os.path import isfile, join
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.decomposition import PCA
from sklearn.preprocessing import StandardScaler, MinMaxScaler
from sklearn.metrics import precision_recall_curve
from sklearn.metrics import average_precision_score
from scipy.special import erf
from scipy.stats import pearsonr, kendalltau, spearmanr
from sklearn import metrics
from sklearn.metrics import roc_curve, auc

In [ ]:
#Training set
d1=pd.read_csv("/home/worldline/data/CSVagg/aggtrx_20150301.csv")
d2=pd.read_csv("/home/worldline/data/CSVagg/aggtrx_20150302.csv")
d3=pd.read_csv("/home/worldline/data/CSVagg/aggtrx_20150303.csv")
d4=pd.read_csv("/home/worldline/data/CSVagg/aggtrx_20150304.csv")
d5=pd.read_csv("/home/worldline/data/CSVagg/aggtrx_20150305.csv")
d6=pd.read_csv("/home/worldline/data/CSVagg/aggtrx_20150306.csv")
d7=pd.read_csv("/home/worldline/data/CSVagg/aggtrx_20150307.csv")

In [ ]:
d=[d1,d2,d3,d4,d5,d5,d6,d7]
df=pd.concat(d, ignore_index=True)
df_f1=df[df.TX_FRAUD==1]
df_f0=df[df.TX_FRAUD==0]


In [ ]:
df_f0_s=df_f0.sample(6124)

In [ ]:
df_sl=[df_f0_s,df_f1]
df_s=pd.concat(df_sl,ignore_index=True)
df_s.info()

In [ ]:
fr=df_s['TX_FRAUD'].apply(str)
age=df_s['AGE']
am=df_s['TX_AMOUNT']
ID=df_s['CARD_PAN_ID']
last=df_s['NB_TRX_LAST_24H']
time_s=df_s['TX_TIME_SECONDS']
new=[ID,am,age,fr,time_s,last]

In [ ]:
training=pd.concat([ID, am, age, fr, time_s, last], axis=1)
training.to_csv("select/training", index=False, header=True)

In [ ]:
t=pd.read_csv("/home/worldline/data/CSVagg/aggtrx_20150314.csv")
testt=pd.concat([t.CARD_PAN_ID,t.TX_AMOUNT,t.AGE,t.TX_FRAUD.apply(str),t.TX_TIME_SECONDS,t.NB_TRX_LAST_24H],axis=1)
testt.to_csv("select/for_test",index=False, header=True)

In [ ]:
t[t.TX_FRAUD==1].info()

In [ ]:
 import h2o

In [ ]:
h2o.init(port=54331)

In [ ]:
h2o.cluster().shutdown()


In [ ]:
col_names=['card_pan_id','tx_amount','age','tx_time_seconds','nb_tx_last_24h']
response_col=''

In [ ]:
df_train = h2o.import_file("select/training")

In [ ]:
df_val=h2o.import_file("select/for_test")

In [ ]:
train_col=['CARD_PAN_ID','TX_AMOUNT','AGE','TX_TIME_SECONDS','NB_TRX_LAST_24H']
resp_col='TX_FRAUD'
type('TX_FRAUD')

In [ ]:
from h2o.estimators.gbm import H2OGradientBoostingEstimator
from h2o.estimators.random_forest import H2ORandomForestEstimator
from h2o.estimators.deeplearning import H2ODeepLearningEstimator
model = H2ORandomForestEstimator(ntrees=50, max_depth=20, nfolds=10, binomial_double_trees=True, stopping_metric= "auc")

In [ ]:
model.train(x=train_col, y=resp_col, training_frame=df_train)

In [ ]:
pred=model.predict(df_val)

In [ ]:
pr=h2o.h2o.as_list(pred, use_pandas=True)
pr.info()
tr=h2o.h2o.as_list(df_val, use_pandas=True)
tr.info()

In [ ]:
# sklearn.metrics.roc_curve(y_true, y_score, pos_label=None, sample_weight=None, drop_intermediate=True)
false_positive_rate, true_positive_rate, thresholds = roc_curve(tr.TX_FRAUD,pr.predict,pos_label=1)
roc_auc = auc(false_positive_rate, true_positive_rate)

In [ ]:
plt.title('ROC_test')
plt.plot(false_positive_rate, true_positive_rate, 'r',
label='AUC = %0.2f'% roc_auc)
plt.legend(loc='lower right')
plt.plot([0,1],[0,1],'g--')
plt.xlim([-0.1,1.2])
plt.ylim([-0.1,1.2])
plt.ylabel('True Positive Rate')
plt.xlabel('False Positive Rate')
plt.show()

In [ ]:
n=[tr.CARD_PAN_ID,pr.predict,tr.TX_FRAUD]
new=pd.concat(n,axis=1)
gr=new.groupby("CARD_PAN_ID").max().sort_values("predict",ascending=False)
top=gr.head(100)
per=(top[top.TX_FRAUD==1].count())/100
perc=per.TX_FRAUD # frauds in top 100
frauds=top[top.TX_FRAUD==1].index.tolist()
top[top.TX_FRAUD==1].count()

In [ ]:
from sklearn.metrics import precision_recall_curve
from sklearn.metrics import average_precision_score


In [ ]:
#sklearn.metrics.precision_recall_curve(y_true, probas_pred, pos_label=None, sample_weight=None)
precision, recall, thresholds=precision_recall_curve(tr.TX_FRAUD,pr.predict,pos_label=1)
#sklearn.metrics.average_precision_score(y_true, y_score, average='macro', sample_weight=None)
average_precision=average_precision_score(tr.TX_FRAUD, pr.predict) 

In [ ]:
from sklearn.multiclass import OneVsRestClassifier
plt.clf()
plt.plot(recall, precision, color='navy',
         label='Precision-Recall')
plt.xlabel('Recall')
plt.ylabel('Precision')
plt.ylim([0.0, 1.05])
plt.xlim([0.0, 1.0])
plt.title('Precision-Recall example: AUC={0:0.2f}'.format(average_precision))
plt.legend(loc="lower left")
plt.show()

In [ ]:
sales = [{'account': 'Jones LLC', 'Jan': 150, 'Feb': 200, 'Mar': 140},
         {'account': 'Alpha Co',  'Jan': 200, 'Feb': 210, 'Mar': 215},
         {'account': 'Blue Inc',  'Jan': 50,  'Feb': 90,  'Mar': 95 },
         {'account': 'Jones LLC', 'Jan': 120, 'Feb': 220, 'Mar': 240},
         {'account': 'Blue Inc',  'Jan': 60,  'Feb': 80,  'Mar': 105 }]
frame = pd.DataFrame(sales)

In [ ]:
k=[1,2,3,4,5,6,7,7,8,9]
for i in k:
    i=i+1
    print(i)

In [ ]:
for i in test:
    pd.concat([i.CARD_PAN_ID,i.TX_AMOUNT,i.AGE,i.TX_FRAUD.apply(str),i.TX_TIME_SECONDS,i.NB_TRX_LAST_24H],axis=1)
    tocsv(i)

In [ ]:
def tocsv(n):
    n.to_csv("select/*",index=False, header=True)

In [ ]:
def reading(files):
    frame_test=[]
    for f in files:
        df=pd.read_csv(f)
        frame_test.append(df)
        del df
    df_final=pd.concat(frame_test,)
    return df_final

In [ ]:
fill=filter(fil, [join(dir_input, f) for f in listdir(dir_input) if isfile(join(dir_input, f))])

In [ ]:
reading(fill)

In [ ]:
import glob

from glob import glob
import os
import h2o

In [ ]:
h2o.init(port=54331)
from h2o.estimators.gbm import H2OGradientBoostingEstimator
from h2o.estimators.random_forest import H2ORandomForestEstimator
from h2o.estimators.deeplearning import H2ODeepLearningEstimator

In [ ]:
from sklearn.metrics import precision_recall_curve
from sklearn.metrics import average_precision_score

In [ ]:
h2o.cluster().shutdown()

In [ ]:
path = r'/home/worldline/data/CSVagg'
filenames =sorted(glob(path + "/*.csv"))
filenames_te1=filenames[filenames.index(path+"/aggtrx_20150314.csv"):(filenames.index(path+"/aggtrx_20150321.csv")+1)]
filenames_te2=filenames[filenames.index(path+"/aggtrx_20150322.csv"):(filenames.index(path+"/aggtrx_20150329.csv")+1)]

In [ ]:
#################################
#preparation data for the testing
for f in filenames_te:
    data=pd.read_csv(f)
    data=pd.concat([data.CARD_PAN_ID,data.TX_AMOUNT,data.AGE,data.TX_FRAUD.apply(str),data.TX_TIME_SECONDS,data.NB_TRX_LAST_24H],axis=1)
    specname = f.split("/")[-1]
    data.to_csv(os.path.join('select/%s'% specname))
    

In [ ]:
#preparation data for training
#filenames_tr=filenames[filenames.index(path+"/aggtrx_20150301.csv"):(filenames.index(path+"/aggtrx_20150307.csv")+1)]
#d=[]
#for k in filenames_tr:
    #data_p=pd.read_csv(k)
    #data=pd.concat([data_p.CARD_PAN_ID,data_p.TX_AMOUNT,data_p.AGE,data_p.TX_FRAUD.apply(str),data_p.TX_TIME_SECONDS,data_p.NB_TRX_LAST_24H],axis=1)
    #d=data
    #df=pd.concat(d, ignore_index=True)
    #print(d)
#df=pd.concat(d, ignore_index=True)
#df_f1=df[df.TX_FRAUD==1]
#df_f0=df[df.TX_FRAUD==0]
#df_f0_s=df_f0.sample(6124)
#df_sl=[df_f0_s,df_f1]
#df_s=pd.concat(df_sl,ignore_index=True)
#data=pd.concat([df_s.CARD_PAN_ID,df_s.TX_AMOUNT,df_s.AGE,df_s.TX_FRAUD.apply(str),df_s.TX_TIME_SECONDS,df_s.NB_TRX_LAST_24H],axis=1)
#specname = k.split("/")[-1]
#data.to_csv(os.path.join('select/train'))
#data.info()

In [ ]:
####################################
new_path=r'select'
names =sorted(glob(new_path + "/*.csv"))
type(names)
names[names.index(new_path+"/aggtrx_20150301.csv")]
names[names.index(new_path+"/aggtrx_20150307.csv")]
training=names[names.index(new_path+"/aggtrx_20150301.csv"):(names.index(new_path+"/aggtrx_20150307.csv")+1)]
test=names[names.index(new_path+"/aggtrx_20150314.csv"):(names.index(new_path+"/aggtrx_20150530.csv")+1)]
#for i in training:
    #df_train = h2o.import_file('select/%s'% specname)
   
    
    


In [ ]:
#testting
path = r'/home/worldline/data/CSVagg'
filenames =sorted(glob(path + "/*.csv"))
filenames_te1=filenames[filenames.index(path+"/aggtrx_20150314.csv"):(filenames.index(path+"/aggtrx_20150321.csv")+1)]
filenames_te2=filenames[filenames.index(path+"/aggtrx_20150322.csv"):(filenames.index(path+"/aggtrx_20150329.csv")+1)]
filenames_te3=filenames[filenames.index(path+"/aggtrx_20150330.csv"):(filenames.index(path+"/aggtrx_20150406.csv")+1)]
filenames_te4=filenames[filenames.index(path+"/aggtrx_20150407.csv"):(filenames.index(path+"/aggtrx_20150414.csv")+1)]
filenames_te5=filenames[filenames.index(path+"/aggtrx_20150415.csv"):(filenames.index(path+"/aggtrx_20150422.csv")+1)]
filenames_te6=filenames[filenames.index(path+"/aggtrx_20150423.csv"):(filenames.index(path+"/aggtrx_20150430.csv")+1)]
filenames_te7=filenames[filenames.index(path+"/aggtrx_20150501.csv"):(filenames.index(path+"/aggtrx_20150508.csv")+1)]
filenames_te8=filenames[filenames.index(path+"/aggtrx_20150509.csv"):(filenames.index(path+"/aggtrx_20150516.csv")+1)]
filenames_te9=filenames[filenames.index(path+"/aggtrx_20150516.csv"):(filenames.index(path+"/aggtrx_20150523.csv")+1)]
filenames_te10=filenames[filenames.index(path+"/aggtrx_20150524.csv"):(filenames.index(path+"/aggtrx_20150531.csv")+1)]
filenames_te=[filenames_te1,filenames_te2,filenames_te3,filenames_te4,filenames_te5,filenames_te6,filenames_te7,filenames_te8,filenames_te9,filenames_te10]


In [ ]:
#Main all variables
ROC_AUC=[]
Average_Precision=[]
Share=[]
Fraud_ID=[]
  h2o.init(port=54331)
    print('The {} is done!'.format(file))
    Final_table=pd.DataFrame({"ROC_AUC": ROC_AUC,
                              "Average_Precision": Average_Precision,
                              "Share": Share})
A_P_m=Final_table.Average_Precision.mean()
AUC_m=Final_table.ROC_AUC.mean()
share_m=Final_table.Share.mean()
print("Average precision mean: {}, ROC AUC mean: {}, Share of fraudulant card in top 100 mean: {}".format(A_P_m, AUC_m, share_m))
Final_table
h2o.cluster().shutdown()

In [ ]:
A_P_m=Final_table.Average_Precision.mean()
AUC_m=Final_table.ROC_AUC.mean()
share_m=Final_table.Share.mean()
print("Average precision mean: {}, ROC AUC mean: {}, Share of fraudulant card in top 100 mean: {}".format(A_P_m, AUC_m, share_m))

In [ ]:
#Main
ROC_AUC=[]
Average_Precision=[]
Share=[]
Fraud_ID=[]
h2o.init(port=54331)
for f in filenames_te1:
    h2o.cluster().shutdown()
    data=pd.read_csv(f)
    data=pd.concat([data.CARD_PAN_ID,data.TX_AMOUNT,data.AGE,data.TX_FRAUD.apply(str),data.TX_TIME_SECONDS,data.NB_TRX_LAST_24H],axis=1)
    data=data[~ data.CARD_PAN_ID.isin(Fraud_ID)]
    specname = f.split("/")[-1]
    data.to_csv(os.path.join('select/%s'% specname))
    h2o.init(port=54331)
    df_train = h2o.import_file("select/training")
    df_val=h2o.import_file(os.path.join('select/%s'% specname))
    train_col=['TX_AMOUNT','AGE','TX_TIME_SECONDS','NB_TRX_LAST_24H']
    resp_col='TX_FRAUD'
    model = H2ORandomForestEstimator(ntrees=25, max_depth=15, nfolds=10, binomial_double_trees=True, stopping_metric= "auc")
    model.train(x=train_col, y=resp_col, training_frame=df_train)
    pred=model.predict(df_val)
    pr=h2o.h2o.as_list(pred, use_pandas=True)
    tr=h2o.h2o.as_list(df_val, use_pandas=True)
    false_positive_rate, true_positive_rate, thresholds = roc_curve(tr.TX_FRAUD,pr.predict,pos_label=1)
    roc_auc = auc(false_positive_rate, true_positive_rate)
    ROC_AUC.append(roc_auc)
    precision, recall, thresholds=precision_recall_curve(tr.TX_FRAUD,pr.predict,pos_label=1)
    average_precision=average_precision_score(tr.TX_FRAUD, pr.predict) 
    Average_Precision.append(average_precision)
    n=[tr.CARD_PAN_ID,pr.predict,tr.TX_FRAUD]
    new=pd.concat(n,axis=1)
    gr=new.groupby("CARD_PAN_ID").max().sort_values("predict",ascending=False)
    top=gr.head(100)
    per=(top[top.TX_FRAUD==1].count())/100
    perc=per.TX_FRAUD # frauds in top 100
    Share.append(perc)
    frauds=top[top.TX_FRAUD==1].index.tolist()
    Fraud_ID=Fraud_ID+list(top[top.TX_FRAUD==1].index.unique())
    
    Final_table=pd.DataFrame({"ROC_AUC": ROC_AUC,
                              "Average_Precision": Average_Precision,
                              "Share": Share})

h2o.cluster().shutdown()
A_P_m=Final_table.Average_Precision.mean()
AUC_m=Final_table.ROC_AUC.mean()
share_m=Final_table.Share.mean()
print("Average precision mean: {}, ROC AUC mean: {}, Share of fraudulant card in top 100 mean: {}".format(A_P_m, AUC_m, share_m))
Final_table
h2o.cluster().shutdown()



In [ ]:
h2o.init(port=54331)
for f in filenames_te2:
    data=pd.read_csv(f)
    data=pd.concat([data.CARD_PAN_ID,data.TX_AMOUNT,data.AGE,data.TX_FRAUD.apply(str),data.TX_TIME_SECONDS,data.NB_TRX_LAST_24H],axis=1)
    data=data[~ data.CARD_PAN_ID.isin(Fraud_ID)]
    specname = f.split("/")[-1]
    data.to_csv(os.path.join('select/%s'% specname))
    df_train = h2o.import_file("select/training")
    df_val=h2o.import_file(os.path.join('select/%s'% specname))
    train_col=['TX_AMOUNT','AGE','TX_TIME_SECONDS','NB_TRX_LAST_24H']
    resp_col='TX_FRAUD'
    model = H2ORandomForestEstimator(ntrees=25, max_depth=15, nfolds=10, binomial_double_trees=True, stopping_metric= "auc")
    model.train(x=train_col, y=resp_col, training_frame=df_train)
    pred=model.predict(df_val)
    pr=h2o.h2o.as_list(pred, use_pandas=True)
    tr=h2o.h2o.as_list(df_val, use_pandas=True)
    false_positive_rate, true_positive_rate, thresholds = roc_curve(tr.TX_FRAUD,pr.predict,pos_label=1)
    roc_auc = auc(false_positive_rate, true_positive_rate)
    ROC_AUC.append(roc_auc)
    precision, recall, thresholds=precision_recall_curve(tr.TX_FRAUD,pr.predict,pos_label=1)
    average_precision=average_precision_score(tr.TX_FRAUD, pr.predict) 
    Average_Precision.append(average_precision)
    n=[tr.CARD_PAN_ID,pr.predict,tr.TX_FRAUD]
    new=pd.concat(n,axis=1)
    gr=new.groupby("CARD_PAN_ID").max().sort_values("predict",ascending=False)
    top=gr.head(100)
    per=(top[top.TX_FRAUD==1].count())/100
    perc=per.TX_FRAUD # frauds in top 100
    Share.append(perc)
    frauds=top[top.TX_FRAUD==1].index.tolist()
    Fraud_ID=Fraud_ID+list(top[top.TX_FRAUD==1].index.unique())
    
Final_table=pd.DataFrame({"ROC_AUC": ROC_AUC,
                          "Average_Precision": Average_Precision,
                          "Share": Share})

h2o.cluster().shutdown()
Final_table.head(14)   


In [ ]:
h2o.init(port=54331)
for f in filenames_te3:
    data=pd.read_csv(f)
    data=pd.concat([data.CARD_PAN_ID,data.TX_AMOUNT,data.AGE,data.TX_FRAUD.apply(str),data.TX_TIME_SECONDS,data.NB_TRX_LAST_24H],axis=1)
    data=data[~ data.CARD_PAN_ID.isin(Fraud_ID)]
    specname = f.split("/")[-1]
    data.to_csv(os.path.join('select/%s'% specname))
    df_train = h2o.import_file("select/training")
    df_val=h2o.import_file(os.path.join('select/%s'% specname))
    train_col=['TX_AMOUNT','AGE','TX_TIME_SECONDS','NB_TRX_LAST_24H']
    resp_col='TX_FRAUD'
    model = H2ORandomForestEstimator(ntrees=25, max_depth=15, nfolds=10, binomial_double_trees=True, stopping_metric= "auc")
    model.train(x=train_col, y=resp_col, training_frame=df_train)
    pred=model.predict(df_val)
    pr=h2o.h2o.as_list(pred, use_pandas=True)
    tr=h2o.h2o.as_list(df_val, use_pandas=True)
    false_positive_rate, true_positive_rate, thresholds = roc_curve(tr.TX_FRAUD,pr.predict,pos_label=1)
    roc_auc = auc(false_positive_rate, true_positive_rate)
    ROC_AUC.append(roc_auc)
    precision, recall, thresholds=precision_recall_curve(tr.TX_FRAUD,pr.predict,pos_label=1)
    average_precision=average_precision_score(tr.TX_FRAUD, pr.predict) 
    Average_Precision.append(average_precision)
    n=[tr.CARD_PAN_ID,pr.predict,tr.TX_FRAUD]
    new=pd.concat(n,axis=1)
    gr=new.groupby("CARD_PAN_ID").max().sort_values("predict",ascending=False)
    top=gr.head(100)
    per=(top[top.TX_FRAUD==1].count())/100
    perc=per.TX_FRAUD # frauds in top 100
    Share.append(perc)
    frauds=top[top.TX_FRAUD==1].index.tolist()
    Fraud_ID=Fraud_ID+list(top[top.TX_FRAUD==1].index.unique())
    
Final_table=pd.DataFrame({"ROC_AUC": ROC_AUC,
                          "Average_Precision": Average_Precision,
                          "Share": Share})

h2o.cluster().shutdown()
Final_table.head(21)   


In [ ]:
h2o.init(port=54331)
for f in filenames_te4:
    data=pd.read_csv(f)
    data=pd.concat([data.CARD_PAN_ID,data.TX_AMOUNT,data.AGE,data.TX_FRAUD.apply(str),data.TX_TIME_SECONDS,data.NB_TRX_LAST_24H],axis=1)
    data=data[~ data.CARD_PAN_ID.isin(Fraud_ID)]
    specname = f.split("/")[-1]
    data.to_csv(os.path.join('select/%s'% specname))
    df_train = h2o.import_file("select/training")
    df_val=h2o.import_file(os.path.join('select/%s'% specname))
    train_col=['TX_AMOUNT','AGE','TX_TIME_SECONDS','NB_TRX_LAST_24H']
    resp_col='TX_FRAUD'
    model = H2ORandomForestEstimator(ntrees=25, max_depth=15, nfolds=10, binomial_double_trees=True, stopping_metric= "auc")
    model.train(x=train_col, y=resp_col, training_frame=df_train)
    pred=model.predict(df_val)
    pr=h2o.h2o.as_list(pred, use_pandas=True)
    tr=h2o.h2o.as_list(df_val, use_pandas=True)
    false_positive_rate, true_positive_rate, thresholds = roc_curve(tr.TX_FRAUD,pr.predict,pos_label=1)
    roc_auc = auc(false_positive_rate, true_positive_rate)
    ROC_AUC.append(roc_auc)
    precision, recall, thresholds=precision_recall_curve(tr.TX_FRAUD,pr.predict,pos_label=1)
    average_precision=average_precision_score(tr.TX_FRAUD, pr.predict) 
    Average_Precision.append(average_precision)
    n=[tr.CARD_PAN_ID,pr.predict,tr.TX_FRAUD]
    new=pd.concat(n,axis=1)
    gr=new.groupby("CARD_PAN_ID").max().sort_values("predict",ascending=False)
    top=gr.head(100)
    per=(top[top.TX_FRAUD==1].count())/100
    perc=per.TX_FRAUD # frauds in top 100
    Share.append(perc)
    frauds=top[top.TX_FRAUD==1].index.tolist()
    Fraud_ID=Fraud_ID+list(top[top.TX_FRAUD==1].index.unique())
    
Final_table=pd.DataFrame({"ROC_AUC": ROC_AUC,
                          "Average_Precision": Average_Precision,
                          "Share": Share})

h2o.cluster().shutdown()
Final_table.head(28)   

In [ ]:
h2o.init(port=54331)
for f in filenames_te5:
    data=pd.read_csv(f)
    data=pd.concat([data.CARD_PAN_ID,data.TX_AMOUNT,data.AGE,data.TX_FRAUD.apply(str),data.TX_TIME_SECONDS,data.NB_TRX_LAST_24H],axis=1)
    data=data[~ data.CARD_PAN_ID.isin(Fraud_ID)]
    specname = f.split("/")[-1]
    data.to_csv(os.path.join('select/%s'% specname))
    df_train = h2o.import_file("select/training")
    df_val=h2o.import_file(os.path.join('select/%s'% specname))
    train_col=['TX_AMOUNT','AGE','TX_TIME_SECONDS','NB_TRX_LAST_24H']
    resp_col='TX_FRAUD'
    model = H2ORandomForestEstimator(ntrees=25, max_depth=15, nfolds=10, binomial_double_trees=True, stopping_metric= "auc")
    model.train(x=train_col, y=resp_col, training_frame=df_train)
    pred=model.predict(df_val)
    pr=h2o.h2o.as_list(pred, use_pandas=True)
    tr=h2o.h2o.as_list(df_val, use_pandas=True)
    false_positive_rate, true_positive_rate, thresholds = roc_curve(tr.TX_FRAUD,pr.predict,pos_label=1)
    roc_auc = auc(false_positive_rate, true_positive_rate)
    ROC_AUC.append(roc_auc)
    precision, recall, thresholds=precision_recall_curve(tr.TX_FRAUD,pr.predict,pos_label=1)
    average_precision=average_precision_score(tr.TX_FRAUD, pr.predict) 
    Average_Precision.append(average_precision)
    n=[tr.CARD_PAN_ID,pr.predict,tr.TX_FRAUD]
    new=pd.concat(n,axis=1)
    gr=new.groupby("CARD_PAN_ID").max().sort_values("predict",ascending=False)
    top=gr.head(100)
    per=(top[top.TX_FRAUD==1].count())/100
    perc=per.TX_FRAUD # frauds in top 100
    Share.append(perc)
    frauds=top[top.TX_FRAUD==1].index.tolist()
    Fraud_ID=Fraud_ID+list(top[top.TX_FRAUD==1].index.unique())
    
Final_table=pd.DataFrame({"ROC_AUC": ROC_AUC,
                          "Average_Precision": Average_Precision,
                          "Share": Share})

h2o.cluster().shutdown()
Final_table.head(35)   

In [ ]:
h2o.init(port=54331)
for f in filenames_te6:
    data=pd.read_csv(f)
    data=pd.concat([data.CARD_PAN_ID,data.TX_AMOUNT,data.AGE,data.TX_FRAUD.apply(str),data.TX_TIME_SECONDS,data.NB_TRX_LAST_24H],axis=1)
    data=data[~ data.CARD_PAN_ID.isin(Fraud_ID)]
    specname = f.split("/")[-1]
    data.to_csv(os.path.join('select/%s'% specname))
    df_train = h2o.import_file("select/training")
    df_val=h2o.import_file(os.path.join('select/%s'% specname))
    train_col=['TX_AMOUNT','AGE','TX_TIME_SECONDS','NB_TRX_LAST_24H']
    resp_col='TX_FRAUD'
    model = H2ORandomForestEstimator(ntrees=25, max_depth=15, nfolds=10, binomial_double_trees=True, stopping_metric= "auc")
    model.train(x=train_col, y=resp_col, training_frame=df_train)
    pred=model.predict(df_val)
    pr=h2o.h2o.as_list(pred, use_pandas=True)
    tr=h2o.h2o.as_list(df_val, use_pandas=True)
    false_positive_rate, true_positive_rate, thresholds = roc_curve(tr.TX_FRAUD,pr.predict,pos_label=1)
    roc_auc = auc(false_positive_rate, true_positive_rate)
    ROC_AUC.append(roc_auc)
    precision, recall, thresholds=precision_recall_curve(tr.TX_FRAUD,pr.predict,pos_label=1)
    average_precision=average_precision_score(tr.TX_FRAUD, pr.predict) 
    Average_Precision.append(average_precision)
    n=[tr.CARD_PAN_ID,pr.predict,tr.TX_FRAUD]
    new=pd.concat(n,axis=1)
    gr=new.groupby("CARD_PAN_ID").max().sort_values("predict",ascending=False)
    top=gr.head(100)
    per=(top[top.TX_FRAUD==1].count())/100
    perc=per.TX_FRAUD # frauds in top 100
    Share.append(perc)
    frauds=top[top.TX_FRAUD==1].index.tolist()
    Fraud_ID=Fraud_ID+list(top[top.TX_FRAUD==1].index.unique())
    
Final_table=pd.DataFrame({"ROC_AUC": ROC_AUC,
                          "Average_Precision": Average_Precision,
                          "Share": Share})

h2o.cluster().shutdown()
Final_table.head(42)   

In [ ]:
h2o.init(port=54331)
for f in filenames_te7:
    data=pd.read_csv(f)
    data=pd.concat([data.CARD_PAN_ID,data.TX_AMOUNT,data.AGE,data.TX_FRAUD.apply(str),data.TX_TIME_SECONDS,data.NB_TRX_LAST_24H],axis=1)
    data=data[~ data.CARD_PAN_ID.isin(Fraud_ID)]
    specname = f.split("/")[-1]
    data.to_csv(os.path.join('select/%s'% specname))
    df_train = h2o.import_file("select/training")
    df_val=h2o.import_file(os.path.join('select/%s'% specname))
    train_col=['TX_AMOUNT','AGE','TX_TIME_SECONDS','NB_TRX_LAST_24H']
    resp_col='TX_FRAUD'
    model = H2ORandomForestEstimator(ntrees=25, max_depth=15, nfolds=10, binomial_double_trees=True, stopping_metric= "auc")
    model.train(x=train_col, y=resp_col, training_frame=df_train)
    pred=model.predict(df_val)
    pr=h2o.h2o.as_list(pred, use_pandas=True)
    tr=h2o.h2o.as_list(df_val, use_pandas=True)
    false_positive_rate, true_positive_rate, thresholds = roc_curve(tr.TX_FRAUD,pr.predict,pos_label=1)
    roc_auc = auc(false_positive_rate, true_positive_rate)
    ROC_AUC.append(roc_auc)
    precision, recall, thresholds=precision_recall_curve(tr.TX_FRAUD,pr.predict,pos_label=1)
    average_precision=average_precision_score(tr.TX_FRAUD, pr.predict) 
    Average_Precision.append(average_precision)
    n=[tr.CARD_PAN_ID,pr.predict,tr.TX_FRAUD]
    new=pd.concat(n,axis=1)
    gr=new.groupby("CARD_PAN_ID").max().sort_values("predict",ascending=False)
    top=gr.head(100)
    per=(top[top.TX_FRAUD==1].count())/100
    perc=per.TX_FRAUD # frauds in top 100
    Share.append(perc)
    frauds=top[top.TX_FRAUD==1].index.tolist()
    Fraud_ID=Fraud_ID+list(top[top.TX_FRAUD==1].index.unique())
    
Final_table=pd.DataFrame({"ROC_AUC": ROC_AUC,
                          "Average_Precision": Average_Precision,
                          "Share": Share})

h2o.cluster().shutdown()
Final_table.head(49)   

In [ ]:
h2o.init(port=54331)
for f in filenames_te8:
    data=pd.read_csv(f)
    data=pd.concat([data.CARD_PAN_ID,data.TX_AMOUNT,data.AGE,data.TX_FRAUD.apply(str),data.TX_TIME_SECONDS,data.NB_TRX_LAST_24H],axis=1)
    data=data[~ data.CARD_PAN_ID.isin(Fraud_ID)]
    specname = f.split("/")[-1]
    data.to_csv(os.path.join('select/%s'% specname))
    df_train = h2o.import_file("select/training")
    df_val=h2o.import_file(os.path.join('select/%s'% specname))
    train_col=['TX_AMOUNT','AGE','TX_TIME_SECONDS','NB_TRX_LAST_24H']
    resp_col='TX_FRAUD'
    model = H2ORandomForestEstimator(ntrees=25, max_depth=15, nfolds=10, binomial_double_trees=True, stopping_metric= "auc")
    model.train(x=train_col, y=resp_col, training_frame=df_train)
    pred=model.predict(df_val)
    pr=h2o.h2o.as_list(pred, use_pandas=True)
    tr=h2o.h2o.as_list(df_val, use_pandas=True)
    false_positive_rate, true_positive_rate, thresholds = roc_curve(tr.TX_FRAUD,pr.predict,pos_label=1)
    roc_auc = auc(false_positive_rate, true_positive_rate)
    ROC_AUC.append(roc_auc)
    precision, recall, thresholds=precision_recall_curve(tr.TX_FRAUD,pr.predict,pos_label=1)
    average_precision=average_precision_score(tr.TX_FRAUD, pr.predict) 
    Average_Precision.append(average_precision)
    n=[tr.CARD_PAN_ID,pr.predict,tr.TX_FRAUD]
    new=pd.concat(n,axis=1)
    gr=new.groupby("CARD_PAN_ID").max().sort_values("predict",ascending=False)
    top=gr.head(100)
    per=(top[top.TX_FRAUD==1].count())/100
    perc=per.TX_FRAUD # frauds in top 100
    Share.append(perc)
    frauds=top[top.TX_FRAUD==1].index.tolist()
    Fraud_ID=Fraud_ID+list(top[top.TX_FRAUD==1].index.unique())
    
Final_table=pd.DataFrame({"ROC_AUC": ROC_AUC,
                          "Average_Precision": Average_Precision,
                          "Share": Share})

h2o.cluster().shutdown()
Final_table.head(56)   

In [ ]:
h2o.init(port=54331)
for f in filenames_te9:
    data=pd.read_csv(f)
    data=pd.concat([data.CARD_PAN_ID,data.TX_AMOUNT,data.AGE,data.TX_FRAUD.apply(str),data.TX_TIME_SECONDS,data.NB_TRX_LAST_24H],axis=1)
    data=data[~ data.CARD_PAN_ID.isin(Fraud_ID)]
    specname = f.split("/")[-1]
    data.to_csv(os.path.join('select/%s'% specname))
    df_train = h2o.import_file("select/training")
    df_val=h2o.import_file(os.path.join('select/%s'% specname))
    train_col=['TX_AMOUNT','AGE','TX_TIME_SECONDS','NB_TRX_LAST_24H']
    resp_col='TX_FRAUD'
    model = H2ORandomForestEstimator(ntrees=25, max_depth=15, nfolds=10, binomial_double_trees=True, stopping_metric= "auc")
    model.train(x=train_col, y=resp_col, training_frame=df_train)
    pred=model.predict(df_val)
    pr=h2o.h2o.as_list(pred, use_pandas=True)
    tr=h2o.h2o.as_list(df_val, use_pandas=True)
    false_positive_rate, true_positive_rate, thresholds = roc_curve(tr.TX_FRAUD,pr.predict,pos_label=1)
    roc_auc = auc(false_positive_rate, true_positive_rate)
    ROC_AUC.append(roc_auc)
    precision, recall, thresholds=precision_recall_curve(tr.TX_FRAUD,pr.predict,pos_label=1)
    average_precision=average_precision_score(tr.TX_FRAUD, pr.predict) 
    Average_Precision.append(average_precision)
    n=[tr.CARD_PAN_ID,pr.predict,tr.TX_FRAUD]
    new=pd.concat(n,axis=1)
    gr=new.groupby("CARD_PAN_ID").max().sort_values("predict",ascending=False)
    top=gr.head(100)
    per=(top[top.TX_FRAUD==1].count())/100
    perc=per.TX_FRAUD # frauds in top 100
    Share.append(perc)
    frauds=top[top.TX_FRAUD==1].index.tolist()
    Fraud_ID=Fraud_ID+list(top[top.TX_FRAUD==1].index.unique())
    
Final_table=pd.DataFrame({"ROC_AUC": ROC_AUC,
                          "Average_Precision": Average_Precision,
                          "Share": Share})

h2o.cluster().shutdown()
Final_table.head(63)   

In [ ]:
h2o.init(port=54331)
for f in filenames_te10:
    data=pd.read_csv(f)
    data=pd.concat([data.CARD_PAN_ID,data.TX_AMOUNT,data.AGE,data.TX_FRAUD.apply(str),data.TX_TIME_SECONDS,data.NB_TRX_LAST_24H],axis=1)
    data=data[~ data.CARD_PAN_ID.isin(Fraud_ID)]
    specname = f.split("/")[-1]
    data.to_csv(os.path.join('select/%s'% specname))
    df_train = h2o.import_file("select/training")
    df_val=h2o.import_file(os.path.join('select/%s'% specname))
    train_col=['TX_AMOUNT','AGE','TX_TIME_SECONDS','NB_TRX_LAST_24H']
    resp_col='TX_FRAUD'
    model = H2ORandomForestEstimator(ntrees=25, max_depth=15, nfolds=10, binomial_double_trees=True, stopping_metric= "auc")
    model.train(x=train_col, y=resp_col, training_frame=df_train)
    pred=model.predict(df_val)
    pr=h2o.h2o.as_list(pred, use_pandas=True)
    tr=h2o.h2o.as_list(df_val, use_pandas=True)
    false_positive_rate, true_positive_rate, thresholds = roc_curve(tr.TX_FRAUD,pr.predict,pos_label=1)
    roc_auc = auc(false_positive_rate, true_positive_rate)
    ROC_AUC.append(roc_auc)
    precision, recall, thresholds=precision_recall_curve(tr.TX_FRAUD,pr.predict,pos_label=1)
    average_precision=average_precision_score(tr.TX_FRAUD, pr.predict) 
    Average_Precision.append(average_precision)
    n=[tr.CARD_PAN_ID,pr.predict,tr.TX_FRAUD]
    new=pd.concat(n,axis=1)
    gr=new.groupby("CARD_PAN_ID").max().sort_values("predict",ascending=False)
    top=gr.head(100)
    per=(top[top.TX_FRAUD==1].count())/100
    perc=per.TX_FRAUD # frauds in top 100
    Share.append(perc)
    frauds=top[top.TX_FRAUD==1].index.tolist()
    Fraud_ID=Fraud_ID+list(top[top.TX_FRAUD==1].index.unique())
    
Final_table=pd.DataFrame({"ROC_AUC": ROC_AUC,
                          "Average_Precision": Average_Precision,
                          "Share": Share})

h2o.cluster().shutdown()
Final_table.head(70)

In [ ]:
plt.plot(Final_table.ROC_AUC)


In [ ]:
plt.plot(Final_table.Average_Precision)

In [ ]:
A_P_m=Final_table.Average_Precision.mean()
AUC_m=Final_table.ROC_AUC.mean()
share_m=Final_table.Share.mean()
print(A_P_m,AUC_m,share_m)
print("Average precision mean: {}, ROC AUC mean: {}, Share of fraudulant card in top 100 mean: {}".format(A_P_m, AUC_m, share_m))

In [ ]:
result=[]
lis=[1,2,3,4,5,6,7,8,9,0]
for i in lis:
    k=i*2
    result.append(k)
print(result)


In [ ]:
plt.plot(Final_table.ROC_AUC)

In [ ]:
Final_table